In [96]:
import numpy as np
from typing import Union
import math

In [172]:
def calculate_Hausholder(v: np.array):
    """
    Helper function to calculate the Hausholder matrix from v.
    
    Parameters
    ----------
    v : np.array
        The given vector. Needs to be normalized prior to passing.
        
    Returns
    -------
    H : np.ndarray
        The Hausholder matrix of v.
    """
    H = np.identity(max(v.shape)) - 2*np.outer(v, v)
    return H


def hausholder_qr(A: np.ndarray):
    """
        Returns the QR factorization obtained through 
        the Hausholder Reflection method. 
        
        Elements belonging to and above diagonal are R, 
        
        Below the diagonal are the column vectors v_1, ..., v_n-1
        which are used to get Q_1, ..., Q_n-1. 
        
    """
    m, n = A.shape
    v_mat = []
    for i in range(n-1):
        
        # Obtaining v_i
        u = A[i:, i].copy()
        u[0] = u[0] + np.copysign(np.linalg.norm(u, ord = 2), u[0]) 
        v =  u/ np.linalg.norm(u, ord = 2)
        
        v_mat.append(v.copy())

        #Calculating Q only to get next iteration
        Q = np.identity(m)
        Q[i:, i:] = calculate_Hausholder(v)
        
        A = np.matmul(Q, A)
        print(A)
    
    A = np.vstack((A, np.zeros(A.shape[1])))
    #Storing v_i's
    for i in range(n-1):
        A[i+1:, i] = np.array(v_mat[i])
        
    return A

def recover_Qis(A: np.ndarray):
    """
    Calculates all Q_is from a Hausholder applied matrix A.
    
    Parameters
    ----------
    
    A : np.ndarray
        The matrix on which Hausholder QR has been performed
        
    Returns
    -------
    Qis : list(np.ndarray)
        A list containing all Q's 
    """
    Qis = []
    
    m, n = A.shape
    
    for i in range(n - 1):
        #v = np.zeros(m - i)
        v = A[i+1:, i].copy()
        #v[:-1] = A[i+1:, i].copy()
        #v[-1] = np.sqrt(1 - np.linalg.norm(v, ord = 2)**2)
        Q = np.identity(m-1)
        Q[i:, i:] = calculate_Hausholder(v)
        Qis.append(Q)
    
    return Qis

def calculate_Q(A: np.ndarray):
    """
    Returns the orthonormal matrix Q from the matrix A after Hausholder QR 
    is applied.
    
    Parameters
    ----------
    
    A : np.ndarray
        The matrix on which Hausholder QR has been performed

    Returns
    -------    
    
    Q: np.ndarray
        The orthonormal matrix involved in QR
    """
    
    Qis = recover_Qis(A.copy())
    Q = np.identity(Qis[0].shape[0])
    for Qi in Qis:
        Q = np.matmul(Q, Qi.T)
        
    return Q

In [173]:
A = np.array([[12, -51, 4, 12],
                [6, 167, -68, 43],
                [-4, 24, -41, 85],
                [1, 0.1, -6, 451]])
HH = hausholder_qr(A)
R_HH = np.triu(HH)
R_HH = np.array([x for x in R_HH if x.any() != 0])
Q_HH = calculate_Q(HH)

Q_C, R_C = np.linalg.qr(A)

[[-1.40356688e+01 -2.09537574e+01  1.43919041e+01 -3.65497366e+01]
 [-4.33680869e-16  1.73924249e+02 -6.56051537e+01  3.18115638e+01]
 [ 2.53269627e-16  1.93838340e+01 -4.25965642e+01  9.24589575e+01]
 [-1.11022302e-16  1.25404151e+00 -5.60085895e+00  4.49135261e+02]]
[[-1.40356688e+01 -2.09537574e+01  1.43919041e+01 -3.65497366e+01]
 [ 4.03744344e-16 -1.75005571e+02  6.99579758e+01 -4.50742546e+01]
 [ 2.99790476e-16 -8.43098429e-16 -3.50657284e+01  8.81877778e+01]
 [-1.08012626e-16 -2.74609209e-16 -5.11364983e+00  4.48858936e+02]]
[[-1.40356688e+01 -2.09537574e+01  1.43919041e+01 -3.65497366e+01]
 [ 4.03744344e-16 -1.75005571e+02  6.99579758e+01 -4.50742546e+01]
 [-2.81066025e-16  8.73901275e-16  3.54366296e+01 -1.52036922e+02]
 [-1.50143085e-16 -1.50072449e-16 -7.25076228e-16  4.31435050e+02]]


In [152]:
[[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [-2.77555756e-16  1.73923077e+02 -6.56923077e+01]
 [ 4.44089210e-16  1.93846154e+01 -4.25384615e+01]]
[[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [ 2.26656301e-16 -1.75000000e+02  7.00000000e+01]
 [ 4.72100991e-16 -7.43126796e-16 -3.50000000e+01]]
[[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [ 2.26656301e-16 -1.75000000e+02  7.00000000e+01]
 [-4.72100991e-16  7.43126796e-16  3.50000000e+01]]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3328549005.py, line 1)

In [169]:
print(Q_C) 
print(Q_HH)

[[-0.8549646   0.39378587 -0.3172958  -0.1152892 ]
 [-0.4274823  -0.90307205  0.03751419 -0.01767618]
 [ 0.2849882  -0.17126068 -0.93463945 -0.12609757]
 [-0.07124705  0.00795914 -0.15609341  0.98513723]]
[[-0.8549646   0.39378587 -0.3172958  -0.1152892 ]
 [-0.4274823  -0.90307205  0.03751419 -0.01767618]
 [ 0.2849882  -0.17126068 -0.93463945 -0.12609757]
 [-0.07124705  0.00795914 -0.15609341  0.98513723]]


In [174]:
abs(Q_C - Q_HH) < 0.001

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [175]:
abs(R_C - R_HH) < 0.01

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [60]:
#From Stack overflow, 
#https://stackoverflow.com/questions/53489237/how-can-you-implement-householder-based-qr-decomposition-in-python
def householder_vectorized(a):
    """Use this version of householder to reproduce the output of np.linalg.qr 
    exactly (specifically, to match the sign convention it uses)
    
    based on https://rosettacode.org/wiki/QR_decomposition#Python
    """
    v = a / (a[0] + np.copysign(np.linalg.norm(a), a[0]))
    v[0] = 1
    tau = 2 / (v.T @ v)
    
    return v,tau

def qr_decomposition(A: np.ndarray) -> Union[np.ndarray, np.ndarray]:
    m,n = A.shape
    R = A.copy()
    Q = np.identity(m)
    
    for j in range(0, n):
        # Apply Householder transformation.
        v, tau = householder_vectorized(R[j:, j, np.newaxis])
        
        H = np.identity(m)
        H[j:, j:] -= tau * (v @ v.T)
        R = H @ R
        Q = H @ Q
        print(R)
    return Q[:n].T, np.triu(R[:n])

In [61]:
A = np.array([[12, -51, 4],
                [6, 167, -68],
                [-4, 24, -41]])
Q, R = qr_decomposition(A)

[[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [-2.77555756e-16  1.73923077e+02 -6.56923077e+01]
 [ 4.44089210e-16  1.93846154e+01 -4.25384615e+01]]
[[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [ 2.26656301e-16 -1.75000000e+02  7.00000000e+01]
 [ 4.72100991e-16 -7.43126796e-16 -3.50000000e+01]]
[[-1.40000000e+01 -2.10000000e+01  1.40000000e+01]
 [ 2.26656301e-16 -1.75000000e+02  7.00000000e+01]
 [-4.72100991e-16  7.43126796e-16  3.50000000e+01]]


In [38]:
print(R_C)
print(R_HH)
print(R)

[[ -14.  -21.   14.]
 [   0. -175.   70.]
 [   0.    0.  -35.]]
[[ 14.  21. -14.]
 [  0. 175. -70.]
 [  0.   0.  35.]]
[[ -14.  -21.   14.]
 [   0. -175.   70.]
 [   0.    0.   35.]]


In [49]:
print(Q_C) 
print(Q)

[[-0.85714286  0.39428571  0.33142857]
 [-0.42857143 -0.90285714 -0.03428571]
 [ 0.28571429 -0.17142857  0.94285714]]
[[-0.85714286  0.39428571 -0.33142857]
 [-0.42857143 -0.90285714  0.03428571]
 [ 0.28571429 -0.17142857 -0.94285714]]
